# 6. Challenge Solution - A2C

So far, we managed to implement the REINFORCE agent version for the Cartpole environment. Now let's try to do the A2C one. As both models have a pretty similar structure and functioning mechanism, this notebook will contain more missing parts.

### Theory

As it has been mentioned in Actor-Critic methods notebook, A2C algorithm has pretty similar expected return expression - the only difference is that the return function is exchanged by the advantage function.
$$
\nabla_{\theta}J(\theta)=\sum_{t = 0}^T\nabla_{\theta}log\pi_{\theta}(a_t|s_t)A(s_t, a_t)
$$
$$
A(s_t, a_t) = Q(s_t, a_t) - V(s_t)
$$

From the structural perspective, A2C model has two parts - actor and critic. Actor takes state as an input and outputs probability distribution for actions, while critic calculated values for those actions.

Your implementation should take the following form:
1. Extracting environment information (state, action, etc.)
2. Passing state through the model to generate action and critic outputs
3. Sample action from the probability distribution
4. Calculating rewards
5. Comparing rewards after taken trajectory to those calculated at the start of the trajectory to generate loss

### Building model

In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
seed = 42
# Discount value
gamma = 0.99
max_steps_per_episode = 10000

# Create the environment
env = gym.make("CartPole-v0")
env.seed(seed)
eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

/Users/lukeleeai/opt/anaconda3/envs/tf2/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


### Defining model

As it has been mentioned, A2C algorithm contains two parts - actor and critic - both of which should be modeled by separate neural networks. In our implementation, the input layer is shared.

In [3]:
num_states = env.observation_space.shape[0]
num_hidden = 128
num_inputs = 4
num_actions = env.action_space.n

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
action = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])

In [4]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

Unfortunately, rendering the game display in Jupyter Notebook requires a complex workaround. If you want to render it, copy the entire code and run it in your favorite IDE like PyCharm and uncomment the line `env.render()`

In [ ]:
while True:  # Run until solved
    state = env.reset()
    episode_reward = 0
    with tf.GradientTape() as tape:
        for timestep in range(1, max_steps_per_episode):
            # env.render()  # Uncomment this line to render the display (doesn't work for Jupyter Notebook)

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            # Apply the sampled action in our environment
            state, reward, done, _ = env.step(action)
            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 6.51 at episode 10
running reward: 9.25 at episode 20
running reward: 10.83 at episode 30
running reward: 11.90 at episode 40
running reward: 14.54 at episode 50
running reward: 18.99 at episode 60
running reward: 22.05 at episode 70
running reward: 19.99 at episode 80
running reward: 19.64 at episode 90
running reward: 17.03 at episode 100
running reward: 16.06 at episode 110
running reward: 16.20 at episode 120
running reward: 15.24 at episode 130
running reward: 14.72 at episode 140
running reward: 16.43 at episode 150
running reward: 20.27 at episode 160
running reward: 28.29 at episode 170
running reward: 36.02 at episode 180
running reward: 32.30 at episode 190
running reward: 46.09 at episode 200
running reward: 55.34 at episode 210
running reward: 56.83 at episode 220
running reward: 44.71 at episode 230
running reward: 43.34 at episode 240
